# Temporal Forecasting 

### Importing the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

### Importing the necessary datasets

In [ ]:
raw=pd.read_csv('../data/raw/ev_charging_patterns.csv')

In [ ]:
processed_df=pd.read_csv('../data/processed/cleaned_charging_patterns.csv')

In [ ]:
raw.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [ ]:
processed_df.head()

,user_id,vehicle_model,battery_capacity_(kwh),charging_station_id,charging_station_location,charging_start_time,charging_end_time,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),...,time_of_day,day_of_week,state_of_charge_(start_%),state_of_charge_(end_%),distance_driven_(since_last_charge)_(km),temperature_(°c),vehicle_age_(years),charger_type,user_type,state_extracted
0,-1.730739,0,1.623166,1.030207,1,-1.178850,-1.181028,0.839652,-1.192647,0.735299,...,1,5,-0.836072,0.636025,1.676333,0.858731,-0.699142,0,1,Houston
1,-1.728115,2,1.235132,1.287279,4,-1.178644,-1.180489,-1.404045,0.658375,0.369203,...,2,1,-1.625199,0.577471,-0.495484,-0.085704,-0.266683,1,0,San Francisco
2,-1.725491,1,0.022525,-0.428847,4,-1.178438,-1.180186,-1.083517,-0.267136,0.149545,...,2,4,-1.749798,-0.300845,-0.972807,0.386513,-0.699142,2,1,San Francisco
3,-1.722866,2,-1.190081,0.585543,1,-1.178232,-1.179787,1.663867,-1.192647,0.515641,...,1,2,1.406710,1.455786,0.554625,1.533328,-1.131602,1,2,Houston
4,-1.720242,2,-1.190081,-0.936043,2,-1.178026,-1.179984,-1.037727,-0.267136,-1.168403,...,2,2,0.202253,-0.652172,0.602358,-1.569817,-1.131602,1,2,Los Angeles


In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   User ID                                   1320 non-null   object 
 1   Vehicle Model                             1320 non-null   object 
 2   Battery Capacity (kWh)                    1320 non-null   float64
 3   Charging Station ID                       1320 non-null   object 
 4   Charging Station Location                 1320 non-null   object 
 5   Charging Start Time                       1320 non-null   object 
 6   Charging End Time                         1320 non-null   object 
 7   Energy Consumed (kWh)                     1254 non-null   float64
 8   Charging Duration (hours)                 1320 non-null   float64
 9   Charging Rate (kW)                        1254 non-null   float64
 10  Charging Cost (USD)                 

In [ ]:
raw.describe()

,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years)
count,1320.000000,1254.000000,1320.000000,1254.000000,1320.000000,1320.000000,1320.000000,1254.000000,1320.000000,1320.000000
mean,74.534692,42.642894,2.269377,25.963003,22.551352,49.130012,75.141590,153.596788,15.263591,3.612843
std,20.626914,22.411705,1.061037,14.011326,10.751494,24.074134,17.080580,86.004987,14.831216,2.309824
min,1.532807,0.045772,0.095314,1.472549,0.234317,2.325959,7.604224,0.862361,-10.724770,0.000000
25%,62.000000,23.881193,1.397623,13.856583,13.368141,27.786903,62.053266,79.445335,2.800664,2.000000
50%,75.000000,42.691405,2.258136,25.603799,22.076360,48.241771,75.682496,152.259867,14.630846,4.000000
75%,85.000000,61.206218,3.112806,37.502998,31.646044,69.277921,88.201370,226.073284,27.981810,6.000000
max,193.003074,152.238758,7.635145,97.342255,69.407743,152.489761,177.708666,398.364775,73.169588,11.688592


In [ ]:
raw.isnull().sum()

User ID                                      0
Vehicle Model                                0
Battery Capacity (kWh)                       0
Charging Station ID                          0
Charging Station Location                    0
Charging Start Time                          0
Charging End Time                            0
Energy Consumed (kWh)                       66
Charging Duration (hours)                    0
Charging Rate (kW)                          66
Charging Cost (USD)                          0
Time of Day                                  0
Day of Week                                  0
State of Charge (Start %)                    0
State of Charge (End %)                      0
Distance Driven (since last charge) (km)    66
Temperature (°C)                             0
Vehicle Age (years)                          0
Charger Type                                 0
User Type                                    0
dtype: int64

MAking the column names compact

In [ ]:
raw.columns = raw.columns.str.lower().str.replace(' ', '_')
raw.columns

Index(['user_id', 'vehicle_model', 'battery_capacity_(kwh)',
       'charging_station_id', 'charging_station_location',
       'charging_start_time', 'charging_end_time', 'energy_consumed_(kwh)',
       'charging_duration_(hours)', 'charging_rate_(kw)',
       'charging_cost_(usd)', 'time_of_day', 'day_of_week',
       'state_of_charge_(start_%)', 'state_of_charge_(end_%)',
       'distance_driven_(since_last_charge)_(km)', 'temperature_(°c)',
       'vehicle_age_(years)', 'charger_type', 'user_type'],
      dtype='object')

In [ ]:
raw.dtypes

user_id                                      object
vehicle_model                                object
battery_capacity_(kwh)                      float64
charging_station_id                          object
charging_station_location                    object
charging_start_time                          object
charging_end_time                            object
energy_consumed_(kwh)                       float64
charging_duration_(hours)                   float64
charging_rate_(kw)                          float64
charging_cost_(usd)                         float64
time_of_day                                  object
day_of_week                                  object
state_of_charge_(start_%)                   float64
state_of_charge_(end_%)                     float64
distance_driven_(since_last_charge)_(km)    float64
temperature_(°c)                            float64
vehicle_age_(years)                         float64
charger_type                                 object
user_type   

### Converting into the date-time format

In [ ]:
# Convert to datetime format
raw['charging_start_time'] = pd.to_datetime(raw['charging_start_time'])
raw['charging_end_time'] = pd.to_datetime(raw['charging_end_time'])

### Extracting day ,hour, month, year, weekdays kind of information

In [ ]:
raw['start_hour'] = raw['charging_start_time'].dt.hour
raw['start_day'] = raw['charging_start_time'].dt.day
raw['start_month'] = raw['charging_start_time'].dt.month
raw['start_year'] = raw['charging_start_time'].dt.year
raw['weekday'] = raw['charging_start_time'].dt.weekday
raw['is_weekend'] = raw['weekday'].isin([5, 6]).astype(int) 


### Generating Lags from previous info

A lag in time-series analysis refers to the use of past data points to predict future values.

Energy consumed (lag_1): The energy consumed during the previous charging session.

Energy consumed (lag_2): The energy consumed two charging sessions ago.

These lag features are useful because they give the model information about past behavior that might help it predict future behavior (e.g., a higher energy consumption this session could be influenced by a higher consumption last session).

Why Lags are Important:

    * Capturing Temporal Dependencies: In time series forecasting, past behavior often influences future values, making lags essential.

    * Trends & Patterns: Lags can help capture recurring patterns (e.g., high energy consumption during specific times of the day).

In [ ]:
raw['energy_consumed_lag_1'] = raw['energy_consumed_(kwh)'].shift(1)
raw['energy_consumed_lag_2'] = raw['energy_consumed_(kwh)'].shift(2)
raw['charging_duration_lag_1'] = raw['charging_duration_(hours)'].shift(1)
raw['charging_duration_lag_2'] = raw['charging_duration_(hours)'].shift(2)

Generating Rolling Statistics 

A rolling window is a way to perform calculations across a subset of consecutive data points (in this case, a window size of 3) and then shift this window one data point at a time.

Window Size = 3 means that, for each data point, you calculate the mean or standard deviation based on the current data point and the two previous ones (total of 3 points).

The window slides over time, so the calculations for the next data point will use the next 3 data points.

Why Use Rolling Statistics?

Smoothing: Rolling statistics (mean, std, etc.) help smooth out fluctuations and capture trends in the data.

Trend Analysis: It allows the model to learn from the recent behavior over a specific period (like the last 3 data points).

Noise Reduction: Rolling means reduce noise and outliers, making the data less volatile.

In [ ]:
raw['energy_consumed_roll_mean_3'] = raw['energy_consumed_(kwh)'].rolling(window=3).mean()
raw['energy_consumed_roll_std_3'] = raw['energy_consumed_(kwh)'].rolling(window=3).std()
raw['charging_duration_roll_mean_3'] = raw['charging_duration_(hours)'].rolling(window=3).mean()
raw['charging_duration_roll_std_3'] = raw['charging_duration_(hours)'].rolling(window=3).std()

In [ ]:
raw.isnull().sum()

user_id                                       0
vehicle_model                                 0
battery_capacity_(kwh)                        0
charging_station_id                           0
charging_station_location                     0
charging_start_time                           0
charging_end_time                             0
energy_consumed_(kwh)                        66
charging_duration_(hours)                     0
charging_rate_(kw)                           66
charging_cost_(usd)                           0
time_of_day                                   0
day_of_week                                   0
state_of_charge_(start_%)                     0
state_of_charge_(end_%)                       0
distance_driven_(since_last_charge)_(km)     66
temperature_(°c)                              0
vehicle_age_(years)                           0
charger_type                                  0
user_type                                     0
start_hour                              

### Extracting all the new columns and forming a new dataset from it

In [ ]:
temporal_features = raw[['start_hour', 'start_day', 'start_month', 'start_year', 
                            'weekday', 'is_weekend', 'energy_consumed_lag_1', 
                            'energy_consumed_lag_2', 'charging_duration_lag_1', 
                            'charging_duration_lag_2', 'energy_consumed_roll_mean_3', 
                            'energy_consumed_roll_std_3', 'charging_duration_roll_mean_3', 
                            'charging_duration_roll_std_3']]

In [ ]:
temporal_features.dtypes

start_hour                         int32
start_day                          int32
start_month                        int32
start_year                         int32
weekday                            int32
is_weekend                         int64
energy_consumed_lag_1            float64
energy_consumed_lag_2            float64
charging_duration_lag_1          float64
charging_duration_lag_2          float64
energy_consumed_roll_mean_3      float64
energy_consumed_roll_std_3       float64
charging_duration_roll_mean_3    float64
charging_duration_roll_std_3     float64
dtype: object

Converting the other dtypes into Int64

In [ ]:
temporal_features['start_hour']=temporal_features['start_hour'].astype('Int64')
temporal_features['start_day']=temporal_features['start_day'].astype('Int64')
temporal_features['start_month']=temporal_features['start_month'].astype('Int64')
temporal_features['start_year']=temporal_features['start_year'].astype('Int64')
temporal_features['weekday']=temporal_features['weekday'].astype('Int64')
temporal_features['is_weekend']=temporal_features['is_weekend'].astype('Int64')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1911219487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['start_hour']=temporal_features['start_hour'].astype('Int64')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1911219487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['start_day']=temporal_features['start_day'].astype('Int64')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1911219487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Converting the float dtypes into Int64

In [ ]:
temporal_features['charging_duration_lag_1']=temporal_features['charging_duration_lag_1'].round(0).astype('Int64')
temporal_features['charging_duration_lag_2']=temporal_features['charging_duration_lag_2'].round(0).astype('Int64')
temporal_features['energy_consumed_lag_1']=temporal_features['energy_consumed_lag_1'].round(0).astype('Int64')
temporal_features['energy_consumed_lag_2']=temporal_features['energy_consumed_lag_2'].round(0).astype('Int64')
temporal_features['energy_consumed_roll_mean_3']=temporal_features['energy_consumed_roll_mean_3'].round(0).astype('Int64')
temporal_features['energy_consumed_roll_std_3']=temporal_features['energy_consumed_roll_std_3'].round(0).astype('Int64')
temporal_features['charging_duration_roll_mean_3']=temporal_features['charging_duration_roll_mean_3'].round(0).astype('Int64')
temporal_features['charging_duration_roll_std_3']=temporal_features['charging_duration_roll_std_3'].round(0).astype('Int64')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1514273904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['charging_duration_lag_1']=temporal_features['charging_duration_lag_1'].round(0).astype('Int64')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1514273904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['charging_duration_lag_2']=temporal_features['charging_duration_lag_2'].round(0).astype('Int64')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4920\1514273904.p

In [ ]:
temporal_features.isnull().sum()

start_hour                         0
start_day                          0
start_month                        0
start_year                         0
weekday                            0
is_weekend                         0
energy_consumed_lag_1             67
energy_consumed_lag_2             68
charging_duration_lag_1            1
charging_duration_lag_2            2
energy_consumed_roll_mean_3      194
energy_consumed_roll_std_3       194
charging_duration_roll_mean_3      2
charging_duration_roll_std_3       2
dtype: int64

### Scaling the dataset

In [ ]:
scaler = StandardScaler()
temporal_scaled= scaler.fit_transform(temporal_features)

In [ ]:
temporal_df = pd.DataFrame(temporal_scaled, columns=['start_hour', 'start_day', 'start_month', 'start_year', 'weekday', 'is_weekend', 
                                                       'energy_consumed_lag_1','energy_consumed_lag_2', 'charging_duration_lag_1', 
                                                        'charging_duration_lag_2', 'energy_consumed_roll_mean_3', 'energy_consumed_roll_std_3', 
                                                        'charging_duration_roll_mean_3', 'charging_duration_roll_std_3'], index=raw.index)


### Concating the new dataset with processed dataset

In [ ]:
processed_df = pd.concat([processed_df, temporal_df], axis=1)

In [ ]:
processed_df.head()

,user_id,vehicle_model,battery_capacity_(kwh),charging_station_id,charging_station_location,charging_start_time,charging_end_time,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),...,weekday,is_weekend,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,-1.730739,0,1.623166,1.030207,1,-1.178850,-1.181028,0.839652,-1.192647,0.735299,...,-1.490321,-0.612372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.728115,2,1.235132,1.287279,4,-1.178644,-1.180489,-1.404045,0.658375,0.369203,...,-1.490321,-0.612372,0.819919,NaN,-1.192129,NaN,NaN,NaN,NaN,NaN
2,-1.725491,1,0.022525,-0.428847,4,-1.178438,-1.180186,-1.083517,-0.267136,0.149545,...,-1.490321,-0.612372,-1.366900,0.819106,0.659954,-1.191373,-0.860889,0.631834,-0.420515,0.091498
3,-1.722866,2,-1.190081,0.585543,1,-1.178232,-1.179787,1.663867,-1.192647,0.515641,...,-1.490321,-0.612372,-1.054497,-1.368267,-0.266088,0.660313,-0.406643,1.783210,-0.420515,0.091498
4,-1.720242,2,-1.190081,-0.936043,2,-1.178026,-1.179984,-1.037727,-0.267136,-1.168403,...,-1.490321,-0.612372,1.623241,-1.055785,-1.192129,-0.265530,-0.255228,1.573869,-0.420515,0.091498


In [ ]:
processed_df.shape

(1320, 35)

In [ ]:
processed_df.columns

Index(['user_id', 'vehicle_model', 'battery_capacity_(kwh)',
       'charging_station_id', 'charging_station_location',
       'charging_start_time', 'charging_end_time', 'energy_consumed_(kwh)',
       'charging_duration_(hours)', 'charging_rate_(kw)',
       'charging_cost_(usd)', 'time_of_day', 'day_of_week',
       'state_of_charge_(start_%)', 'state_of_charge_(end_%)',
       'distance_driven_(since_last_charge)_(km)', 'temperature_(°c)',
       'vehicle_age_(years)', 'charger_type', 'user_type', 'state_extracted',
       'start_hour', 'start_day', 'start_month', 'start_year', 'weekday',
       'is_weekend', 'energy_consumed_lag_1', 'energy_consumed_lag_2',
       'charging_duration_lag_1', 'charging_duration_lag_2',
       'energy_consumed_roll_mean_3', 'energy_consumed_roll_std_3',
       'charging_duration_roll_mean_3', 'charging_duration_roll_std_3'],
      dtype='object')

In [ ]:
processed_df.columns=processed_df.columns.str.replace('distance_driven_(since_last_charge)_(km)','distance_driven(km)')
processed_df.columns=processed_df.columns.str.replace('temperature_(°c)', 'temp(°c)')
processed_df.columns=processed_df.columns.str.replace('vehicle_age_(years)','vehicle_age')
processed_df.columns=processed_df.columns.str.replace('charging_cost_(usd)','charge_cost(usd)')

In [ ]:
processed_df.isnull().sum()

user_id                            0
vehicle_model                      0
battery_capacity_(kwh)             0
charging_station_id                0
charging_station_location          0
charging_start_time                0
charging_end_time                  0
energy_consumed_(kwh)              0
charging_duration_(hours)          0
charging_rate_(kw)                 0
charge_cost(usd)                   0
time_of_day                        0
day_of_week                        0
state_of_charge_(start_%)          0
state_of_charge_(end_%)            0
distance_driven(km)                0
temp(°c)                           0
vehicle_age                        0
charger_type                       0
user_type                          0
state_extracted                    0
start_hour                         0
start_day                          0
start_month                        0
start_year                         0
weekday                            0
is_weekend                         0
e

Filling the null values

In [ ]:
processed_df.columns

Index(['user_id', 'vehicle_model', 'battery_capacity_(kwh)',
       'charging_station_id', 'charging_station_location',
       'charging_start_time', 'charging_end_time', 'energy_consumed_(kwh)',
       'charging_duration_(hours)', 'charging_rate_(kw)', 'charge_cost(usd)',
       'time_of_day', 'day_of_week', 'state_of_charge_(start_%)',
       'state_of_charge_(end_%)', 'distance_driven(km)', 'temp(°c)',
       'vehicle_age', 'charger_type', 'user_type', 'state_extracted',
       'start_hour', 'start_day', 'start_month', 'start_year', 'weekday',
       'is_weekend', 'energy_consumed_lag_1', 'energy_consumed_lag_2',
       'charging_duration_lag_1', 'charging_duration_lag_2',
       'energy_consumed_roll_mean_3', 'energy_consumed_roll_std_3',
       'charging_duration_roll_mean_3', 'charging_duration_roll_std_3'],
      dtype='object')

In [ ]:
processed_df[temporal_df.columns] = processed_df[temporal_df.columns].fillna(processed_df[temporal_df.columns].median())

In [ ]:
processed_df = processed_df.sort_values(by=['user_id', 'charging_start_time']).reset_index(drop=True)

In [ ]:
processed_df['charging_start_time'] = pd.to_datetime(processed_df['charging_start_time'])

In [ ]:
processed_df.head()

,user_id,vehicle_model,battery_capacity_(kwh),charging_station_id,charging_station_location,charging_start_time,charging_end_time,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),...,weekday,is_weekend,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,-1.730739,0,1.623166,1.030207,1,1969-12-31 23:59:59.999999999,-1.181028,0.839652,-1.192647,0.735299,...,-1.490321,-0.612372,0.016598,0.015581,-0.266088,-0.265530,0.047603,0.003811,-0.420515,0.091498
1,-1.728115,2,1.235132,1.287279,4,1969-12-31 23:59:59.999999999,-1.180489,-1.404045,0.658375,0.369203,...,-1.490321,-0.612372,0.819919,0.015581,-1.192129,-0.265530,0.047603,0.003811,-0.420515,0.091498
2,-1.725491,1,0.022525,-0.428847,4,1969-12-31 23:59:59.999999999,-1.180186,-1.083517,-0.267136,0.149545,...,-1.490321,-0.612372,-1.366900,0.819106,0.659954,-1.191373,-0.860889,0.631834,-0.420515,0.091498
3,-1.722866,2,-1.190081,0.585543,1,1969-12-31 23:59:59.999999999,-1.179787,1.663867,-1.192647,0.515641,...,-1.490321,-0.612372,-1.054497,-1.368267,-0.266088,0.660313,-0.406643,1.783210,-0.420515,0.091498
4,-1.720242,2,-1.190081,-0.936043,2,1969-12-31 23:59:59.999999999,-1.179984,-1.037727,-0.267136,-1.168403,...,-1.490321,-0.612372,1.623241,-1.055785,-1.192129,-0.265530,-0.255228,1.573869,-0.420515,0.091498


Cyclic Time Encoding

In [ ]:
processed_df['start_hour_sin'] = np.sin(2 * np.pi * processed_df['start_hour'] / 24)
processed_df['start_hour_cos'] = np.cos(2 * np.pi * processed_df['start_hour'] / 24)

processed_df['weekday_sin'] = np.sin(2 * np.pi * processed_df['weekday'] / 7)
processed_df['weekday_cos'] = np.cos(2 * np.pi * processed_df['weekday'] / 7)

Gap Interval (Time since last charge)

In [ ]:
processed_df['time_since_last_charge'] = processed_df['charging_start_time'].diff().dt.total_seconds() / 3600

In [ ]:
processed_df.head()

,user_id,vehicle_model,battery_capacity_(kwh),charging_station_id,charging_station_location,charging_start_time,charging_end_time,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),...,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3,start_hour_sin,start_hour_cos,weekday_sin,weekday_cos,time_since_last_charge
0,-1.730739,0,1.623166,1.030207,1,1969-12-31 23:59:59.999999999,-1.181028,0.839652,-1.192647,0.735299,...,-0.265530,0.047603,0.003811,-0.420515,0.091498,-0.421350,0.906898,-0.972958,0.230983,NaN
1,-1.728115,2,1.235132,1.287279,4,1969-12-31 23:59:59.999999999,-1.180489,-1.404045,0.658375,0.369203,...,-0.265530,0.047603,0.003811,-0.420515,0.091498,-0.386758,0.922181,-0.972958,0.230983,0.0
2,-1.725491,1,0.022525,-0.428847,4,1969-12-31 23:59:59.999999999,-1.180186,-1.083517,-0.267136,0.149545,...,-1.191373,-0.860889,0.631834,-0.420515,0.091498,-0.351613,0.936146,-0.972958,0.230983,0.0
3,-1.722866,2,-1.190081,0.585543,1,1969-12-31 23:59:59.999999999,-1.179787,1.663867,-1.192647,0.515641,...,0.660313,-0.406643,1.783210,-0.420515,0.091498,-0.315964,0.948771,-0.972958,0.230983,0.0
4,-1.720242,2,-1.190081,-0.936043,2,1969-12-31 23:59:59.999999999,-1.179984,-1.037727,-0.267136,-1.168403,...,-0.265530,-0.255228,1.573869,-0.420515,0.091498,-0.279864,0.960040,-0.972958,0.230983,0.0


In [ ]:
new_cols=processed_df[['start_hour_sin' , 'start_hour_cos', 'weekday_sin' , 'weekday_cos' , 'time_since_last_charge']]

In [ ]:
scaler = StandardScaler()
scaled= scaler.fit_transform(new_cols)

In [ ]:
processed_df.dtypes

user_id                                 float64
vehicle_model                             int64
battery_capacity_(kwh)                  float64
charging_station_id                     float64
charging_station_location                 int64
charging_start_time              datetime64[ns]
charging_end_time                       float64
energy_consumed_(kwh)                   float64
charging_duration_(hours)               float64
charging_rate_(kw)                      float64
charge_cost(usd)                        float64
time_of_day                               int64
day_of_week                               int64
state_of_charge_(start_%)               float64
state_of_charge_(end_%)                 float64
distance_driven(km)                     float64
temp(°c)                                float64
vehicle_age                             float64
charger_type                              int64
user_type                                 int64
state_extracted                         

In [ ]:
processed_df.isnull().sum()

user_id                          0
vehicle_model                    0
battery_capacity_(kwh)           0
charging_station_id              0
charging_station_location        0
charging_start_time              0
charging_end_time                0
energy_consumed_(kwh)            0
charging_duration_(hours)        0
charging_rate_(kw)               0
charge_cost(usd)                 0
time_of_day                      0
day_of_week                      0
state_of_charge_(start_%)        0
state_of_charge_(end_%)          0
distance_driven(km)              0
temp(°c)                         0
vehicle_age                      0
charger_type                     0
user_type                        0
state_extracted                  0
start_hour                       0
start_day                        0
start_month                      0
start_year                       0
weekday                          0
is_weekend                       0
energy_consumed_lag_1            0
energy_consumed_lag_

In [ ]:
processed_df['time_since_last_charge'] = processed_df['time_since_last_charge'].round(0).astype('Int64')

In [ ]:
processed_df['time_since_last_charge']= processed_df['time_since_last_charge'].fillna(processed_df['time_since_last_charge'].median())

In [ ]:
processed_df.isnull().sum().sum()

np.int64(0)

In [ ]:
processed_df.shape

(1320, 40)

In [ ]:
processed_df.columns

Index(['user_id', 'vehicle_model', 'battery_capacity_(kwh)',
       'charging_station_id', 'charging_station_location',
       'charging_start_time', 'charging_end_time', 'energy_consumed_(kwh)',
       'charging_duration_(hours)', 'charging_rate_(kw)', 'charge_cost(usd)',
       'time_of_day', 'day_of_week', 'state_of_charge_(start_%)',
       'state_of_charge_(end_%)', 'distance_driven(km)', 'temp(°c)',
       'vehicle_age', 'charger_type', 'user_type', 'state_extracted',
       'start_hour', 'start_day', 'start_month', 'start_year', 'weekday',
       'is_weekend', 'energy_consumed_lag_1', 'energy_consumed_lag_2',
       'charging_duration_lag_1', 'charging_duration_lag_2',
       'energy_consumed_roll_mean_3', 'energy_consumed_roll_std_3',
       'charging_duration_roll_mean_3', 'charging_duration_roll_std_3',
       'start_hour_sin', 'start_hour_cos', 'weekday_sin', 'weekday_cos',
       'time_since_last_charge'],
      dtype='object')

In [ ]:
processed_df.isnull().sum().sum()

np.int64(0)

In [ ]:
processed_df.to_csv("../data/processed/temporal_forecast_result.csv", index=False)